In [ ]:
# Install Diffusers
pip install -U diffusers


In [ ]:
# Basic text-to-Image using SD3
import torch
from diffusers import StableDiffusion3Pipeline

# Set your seed value
seed = 123456  # You can use any integer value you want

# Create a generator with the seed
generator = torch.Generator("cuda").manual_seed(seed)

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")
image = pipe(
    "A photo of a group of teddy bears eating pizza on the surface of mars",
    negative_prompt="pepperoni",
    num_inference_steps=40,
    generator=generator  # Add the generator here
).images
image[0].save("teddies.png")

In [ ]:
# Basic Image to Image using SD3
import torch
from diffusers import StableDiffusion3Img2ImgPipeline
from PIL import Image

# Set your seed value
seed = 123456

# Create a generator with the seed
generator = torch.Generator("cuda").manual_seed(seed)

# Load the model
pipe = StableDiffusion3Img2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")

# Load and preprocess the initial image
init_image = Image.open("puppy1.jpg").convert("RGB")

# Generate the image
image = pipe(
    prompt="An highly detailed photograph of a baby dragon",
    image=init_image,
    strength=0.4,
    num_inference_steps=100,
    generator=generator
).images

image[0].save("dragon2.png")

In [ ]:
# Basic Inpainting using SD3
import torch
from diffusers import StableDiffusion3InpaintPipeline
from PIL import Image
import numpy as np

# Load the inpainting pipeline
pipe = StableDiffusion3InpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")

# Set seed for reproducibility
generator = torch.Generator("cuda").manual_seed(42)

# Load the original image and mask
original_image = Image.open("puppy.jpg").convert("RGB")
mask_image = Image.open("puppymask.png").convert("L")  # Mask should be black and white

# Generate the inpainted image
image = pipe(
    prompt="on the surface of the moon",
    image=original_image,
    mask_image=mask_image,
    num_inference_steps=50,
    generator=generator,
    strength=0.99  # How much to inpaint the masked area
).images[0]

# Save the result
image.save("inpainted_result.png")

In [ ]:
# A fun little project to render an image, and grab the intermediate steps into
# a series of images to make a video from
import torch
from diffusers import StableDiffusion3Pipeline
from PIL import Image
import os

def save_intermediate_steps(pipe, step_index: int, timestep: int, callback_kwargs: dict):
    # Get the current latent
    latents = callback_kwargs["latents"]

    # Decode the latents to an image
    with torch.no_grad():
        image = pipe.vae.decode(latents / pipe.vae.config.scaling_factor, return_dict=False)[0]

    # Convert to PIL Image
    image = pipe.image_processor.postprocess(image, output_type="pil")[0]

    # Create output directory if it doesn't exist
    os.makedirs("drive/MyDrive/intermediate_steps", exist_ok=True)

    # Save the image
    image.save(f"drive/MyDrive/intermediate_steps/step_{step_index:02d}.png")

    return callback_kwargs

# Set your seed value
seed = 42
generator = torch.Generator("cuda").manual_seed(seed)

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")

image = pipe(
    "A photo of a group of teddy bears eating pizza on the surface of mars",
    num_inference_steps=100,
    generator=generator,
    callback_on_step_end=save_intermediate_steps
).images[0]

image.save("teddies.png")

In [ ]:
import torch
from diffusers import StableDiffusion3Img2ImgPipeline
from PIL import Image
import os
import numpy as np

# Create output directory if it doesn't exist
output_dir = "drive/MyDrive/dragon_steps"
os.makedirs(output_dir, exist_ok=True)

# Set your seed value
seed = 123456

# Create a generator with the seed
generator = torch.Generator("cuda").manual_seed(seed)

# Load the model
pipe = StableDiffusion3Img2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")

# Load and preprocess the initial image
init_image = Image.open("puppy1.jpg").convert("RGB")

# Create array of strength values from 0.1 to 1.0
strengths = np.arange(0.1, 1.01, 0.01)  # Using 1.01 to include 1.0

# Loop through strengths and generate images
for i, strength in enumerate(strengths, 1):
    # Generate the image
    image = pipe(
        prompt="An highly detailed photograph of a baby dragon",
        image=init_image,
        strength=float(strength),  # Convert numpy float to Python float
        num_inference_steps=100,
        generator=generator
    ).images

    # Save with padded number in filename (dragon01.png, dragon02.png, etc.)
    filename = f"dragon{i:02d}.png"
    filepath = os.path.join(output_dir, filename)
    image[0].save(filepath)

    # Optional: Print progress
    print(f"Generated image {i}/100 with strength {strength:.2f}")